In [ ]:
# 컬럼값동일하게 맞출것!!

In [1]:
import pandas as pd
import pickle
import argparse
import os
import glob
import pydicom
import sys


/tmp/ipykernel_3567064/925965458.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import pandas as pd
import pickle

def load_pickle_file(file_path):
    with open(file_path, 'rb') as file:
        return pickle.load(file)
        
def find_volumes_files(root_path):
    volumes_files = []
    for root, dirs, files in os.walk(root_path):
        for file in files:
            if file == 'volumes.pickle':
                volumes_files.append(os.path.join(root, file))
    return volumes_files

def extract_subject_data(volumes_file_path):
    subject_name = os.path.dirname(volumes_file_path).split(os.sep)[-3]
    volumes_data = load_pickle_file(volumes_file_path)
    volumes_data['subject'] = subject_name
    return volumes_data

def compile_data(volumes_files):
    data = []
    for volumes_file in volumes_files:
        subject_data = extract_subject_data(volumes_file)
        data.append(subject_data)
    df = pd.DataFrame(data)
    # You might need to adjust the DataFrame to ensure it has the structure you want
    return df

root_path = '/home/limseoyoung/drwelly1/SeoyoungLim/drwelly-debug/drwelly_oasis1_before_swift'
volumes_files = find_volumes_files(root_path)
df = compile_data(volumes_files)

In [3]:
df.columns

Index(['total_intracranial_volume', 'unknown', 'left_cerebral_white_matter',
       'left_lateral_ventricle', 'left_inf_lat_vent',
       'left_cerebellum_white_matter', 'left_cerebellum_cortex',
       'left_thalamus', 'left_caudate', 'left_putamen',
       ...
       'cerebellum', 'cerebellum_white_matter', 'cerebellum_cortex',
       'lateral_ventricle', 'inf_lat_vent', 'total_lateral_ventricle',
       'wm_hypointensities', 'HippocampusOccupancyScore',
       'HippocampusAsymmetryIndex', 'subject'],
      dtype='object', length=144)

In [4]:
df['subject'] = df['subject'].str.replace('_Tag_pr100_sr100_ppf100_spf100_ipf100_sipf179', '')


In [5]:
# 'mpr-x'에 따라 4개의 데이터프레임으로 분할
volume_df_mpr1 = df[df['subject'].str.contains('MR1_mpr-1')]
volume_df_mpr2 = df[df['subject'].str.contains('MR1_mpr-2')]
volume_df_mpr3 = df[df['subject'].str.contains('MR1_mpr-3')]
volume_df_mpr4 = df[df['subject'].str.contains('MR1_mpr-4')]

In [6]:

def extract_number(subj):
    parts = subj.split('_')  # '_'를 기준으로 문자열 분리
    if len(parts) > 1:
        number_part = parts[1]  # "OAS1_" 다음 오는 부분을 선택
        # 숫자만 추출
        numbers = ''.join(filter(str.isdigit, number_part))
        return int(numbers) if numbers.isdigit() else None
    return None

In [7]:

volume_df_mpr1 ['sort_key'] = volume_df_mpr1['subject'].apply(extract_number)  # 정렬 키 생성
volume_df_sorted1 = volume_df_mpr1 .sort_values(by='sort_key').drop(columns=['sort_key'])  # 정렬 및 정렬 키 컬럼 삭제


/tmp/ipykernel_3567064/3245932642.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  volume_df_mpr1 ['sort_key'] = volume_df_mpr1['subject'].apply(extract_number)  # 정렬 키 생성


In [8]:

volume_df_mpr2 ['sort_key'] = volume_df_mpr2['subject'].apply(extract_number)  # 정렬 키 생성
volume_df_sorted2 = volume_df_mpr2.sort_values(by='sort_key').drop(columns=['sort_key'])  # 정렬 및 정렬 키 컬럼 삭제


/tmp/ipykernel_3567064/4065747069.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  volume_df_mpr2 ['sort_key'] = volume_df_mpr2['subject'].apply(extract_number)  # 정렬 키 생성


In [9]:

volume_df_mpr3['sort_key'] = volume_df_mpr3['subject'].apply(extract_number)  # 정렬 키 생성
volume_df_sorted3 = volume_df_mpr3.sort_values(by='sort_key').drop(columns=['sort_key'])  # 정렬 및 정렬 키 컬럼 삭제


/tmp/ipykernel_3567064/593579216.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  volume_df_mpr3['sort_key'] = volume_df_mpr3['subject'].apply(extract_number)  # 정렬 키 생성


In [10]:
volume_df_mpr4['sort_key'] = volume_df_mpr4['subject'].apply(extract_number)  # 정렬 키 생성
volume_df_sorted4 = volume_df_mpr4.sort_values(by='sort_key').drop(columns=['sort_key'])  # 정렬 및 정렬 키 컬럼 삭제


/tmp/ipykernel_3567064/3330656496.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  volume_df_mpr4['sort_key'] = volume_df_mpr4['subject'].apply(extract_number)  # 정렬 키 생성


In [11]:

all_data = pd.concat([volume_df_mpr1,volume_df_mpr2,volume_df_mpr3,volume_df_mpr4])
all_data_long = pd.melt(all_data, id_vars=["subject"], var_name="ROI", value_name="Volume")


In [12]:

all_data_long['Subject_ID'] = all_data_long['subject'].str.extract(r'(OAS1_\d+)')
all_data_long['Session'] = all_data_long['subject'].str.extract(r'(mpr-\d)')


In [13]:

re_long= all_data_long.dropna(subset=['Volume'])


grouped = re_long.groupby(['Subject_ID', 'ROI'])['Volume'].agg(['mean', 'var','std']).reset_index()

grouped

,Subject_ID,ROI,mean,var,std
0,OAS1_0001,3rd_ventricle,2.836696e+03,6.260005e+02,25.020001
1,OAS1_0001,4th_ventricle,2.680202e+03,3.907280e+03,62.508238
2,OAS1_0001,HippocampusAsymmetryIndex,1.820663e-02,4.944350e-05,0.007032
3,OAS1_0001,HippocampusOccupancyScore,7.046249e-01,6.866100e-05,0.008286
4,OAS1_0001,accumbens_area,7.613742e+02,3.372898e+02,18.365452
...,...,...,...,...,...
59899,OAS1_0457,total_lateral_ventricle,2.337602e+04,8.021583e+03,89.563288
59900,OAS1_0457,unknown,1.544857e+07,3.046440e+06,1745.405451
59901,OAS1_0457,ventraldc,6.522831e+03,1.826265e+03,42.734825
59902,OAS1_0457,whole_brain,9.287733e+05,3.360636e+07,5797.098950


In [14]:
grouped['CV'] = grouped['std'] / grouped['mean']

In [15]:
roi_mean_cv = grouped.groupby('ROI')['CV'].mean().reset_index()
roi_mean_cv.rename(columns={'CV': 'Mean_CV'}, inplace=True)

roi_mean_cv


,ROI,Mean_CV
0,3rd_ventricle,0.016073
1,4th_ventricle,0.028253
2,HippocampusAsymmetryIndex,0.441199
3,HippocampusOccupancyScore,0.010211
4,accumbens_area,0.022256
...,...,...
139,total_lateral_ventricle,0.006394
140,unknown,0.000192
141,ventraldc,0.008372
142,whole_brain,0.005219


In [15]:
#roi_mean_cv.to_csv("roi_mean_CV_DRwelly_before.csv")

In [17]:
#volume_df_sorted4.shape

(357, 144)

In [34]:
def update_df_columns_to_roi(df, roi_dict):
    columns_to_remove = [] 
    new_columns_data = {}   


    for col in df.columns:
        matched = False
        for roi, sub_columns in roi_dict.items():
      
            if any(col.endswith(sub_col.replace('-', '_')) for sub_col in sub_columns):
                matched = True
                
                if roi not in new_columns_data:
                    new_columns_data[roi] = df[col].fillna(0)
                else:
                    new_columns_data[roi] += df[col].fillna(0)
                
                
                if col not in columns_to_remove:
                    columns_to_remove.append(col)
                break  
        if not matched:
           
            new_columns_data[col] = df[col]

    
    new_df = pd.DataFrame(new_columns_data)

    return new_df


new_df1 = update_df_columns_to_roi(volume_df_mpr1.copy(), combined_roi_display_dict)
new_df2 = update_df_columns_to_roi(volume_df_mpr2.copy(), combined_roi_display_dict)

new_df3 = update_df_columns_to_roi(volume_df_mpr3.copy(), combined_roi_display_dict)
new_df4 = update_df_columns_to_roi(volume_df_mpr4.copy(), combined_roi_display_dict)

NameError: name 'combined_roi_display_dict' is not defined

NameError: name 'new_df1' is not defined

In [97]:
volume_df_mpr1.columns

Index(['total_intracranial_volume', 'unknown', 'left_cerebral_white_matter',
       'left_lateral_ventricle', 'left_inf_lat_vent',
       'left_cerebellum_white_matter', 'left_cerebellum_cortex',
       'left_thalamus', 'left_caudate', 'left_putamen',
       ...
       'cerebellum_white_matter', 'cerebellum_cortex', 'lateral_ventricle',
       'inf_lat_vent', 'total_lateral_ventricle', 'wm_hypointensities',
       'HippocampusOccupancyScore', 'HippocampusAsymmetryIndex', 'subject',
       'sort_key'],
      dtype='object', length=145)

In [102]:
columns_to_include = [
    'csf', '3rd_ventricle', 'inf_lat_vent', '4th_ventricle', 'frontal', 'temporal', 'parietal', 'occipital',
    'insula', 'thalamus', 'caudate', 'putamen', 'pallidum', 'hippocampus', 'amygdala', 'accumbens_area',
    'ventraldc', 'cingulate', 'cerebellum', 'subject', 'brain_stem', 'parietal', 'lateral_ventricle',
    'total_cerebral_white_matter' 
]


def prepare_filtered_df(volume_df):
    
    volume_df['total_cerebral_white_matter'] = volume_df['left_cerebral_white_matter'] + volume_df['right_cerebral_white_matter']
    

    filtered_df = volume_df[columns_to_include].copy()
    return filtered_df


filtered_df1 = prepare_filtered_df(volume_df_mpr1)
filtered_df2 = prepare_filtered_df(volume_df_mpr2)
filtered_df3 = prepare_filtered_df(volume_df_mpr3)
filtered_df4 = prepare_filtered_df(volume_df_mpr4)

/tmp/ipykernel_3567064/145641766.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  volume_df['total_cerebral_white_matter'] = volume_df['left_cerebral_white_matter'] + volume_df['right_cerebral_white_matter']
/tmp/ipykernel_3567064/145641766.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  volume_df['total_cerebral_white_matter'] = volume_df['left_cerebral_white_matter'] + volume_df['right_cerebral_white_matter']
/tmp/ipykernel_3567064/145641766.py:11: SettingWithCopyWarning: 
A value is trying to be

In [103]:
filtered_df1.columns

Index(['csf', '3rd_ventricle', 'inf_lat_vent', '4th_ventricle', 'frontal',
       'temporal', 'parietal', 'occipital', 'insula', 'thalamus', 'caudate',
       'putamen', 'pallidum', 'hippocampus', 'amygdala', 'accumbens_area',
       'ventraldc', 'cingulate', 'cerebellum', 'subject', 'brain_stem',
       'parietal', 'lateral_ventricle', 'total_cerebral_white_matter'],
      dtype='object')

In [87]:
pwd

'/home/limseoyoung/debug'

In [104]:
filtered_df1.to_csv("before_drwelly_MPR1_drwelly_accuracy_20240430_final111111.csv")

In [105]:
filtered_df2.to_csv("before_drwelly_MPR2_drwelly_accuracy_20240430_final111111.csv")

In [106]:
filtered_df3.to_csv("before_drwelly_MPR3_drwelly_accuracy_20240430_final111111.csv")

In [107]:
filtered_df4.to_csv("before_drwelly_MPR4_drwelly_accuracy_20240430_final111111.csv")

In [108]:

all_data = pd.concat([filtered_df1,filtered_df2,filtered_df3,filtered_df4])
all_data_long = pd.melt(all_data, id_vars=["subject"], var_name="ROI", value_name="Volume")


In [109]:

all_data_long['Subject_ID'] = all_data_long['subject'].str.extract(r'(OAS1_\d+)')
all_data_long['Session'] = all_data_long['subject'].str.extract(r'(mpr-\d)')


In [110]:

re_long= all_data_long.dropna(subset=['Volume'])


grouped = re_long.groupby(['Subject_ID', 'ROI'])['Volume'].agg(['mean', 'var','std']).reset_index()

grouped

,Subject_ID,ROI,mean,var,std
0,OAS1_0001,3rd_ventricle,2836.695605,6.260005e+02,25.020001
1,OAS1_0001,4th_ventricle,2680.201863,3.907280e+03,62.508238
2,OAS1_0001,accumbens_area,761.374211,3.372898e+02,18.365452
3,OAS1_0001,amygdala,2562.552869,4.025478e+03,63.446653
4,OAS1_0001,brain_stem,17644.862349,7.908943e+03,88.932241
...,...,...,...,...,...
9147,OAS1_0457,putamen,7960.800927,2.526669e+03,50.265981
9148,OAS1_0457,temporal,94703.895828,3.332205e+06,1825.432770
9149,OAS1_0457,thalamus,13499.907644,3.579510e+03,59.829004
9150,OAS1_0457,total_cerebral_white_matter,361514.435530,9.824032e+04,313.433114


In [111]:
grouped['CV'] = grouped['std'] / grouped['mean']

In [112]:
grouped

,Subject_ID,ROI,mean,var,std,CV
0,OAS1_0001,3rd_ventricle,2836.695605,6.260005e+02,25.020001,0.008820
1,OAS1_0001,4th_ventricle,2680.201863,3.907280e+03,62.508238,0.023322
2,OAS1_0001,accumbens_area,761.374211,3.372898e+02,18.365452,0.024121
3,OAS1_0001,amygdala,2562.552869,4.025478e+03,63.446653,0.024759
4,OAS1_0001,brain_stem,17644.862349,7.908943e+03,88.932241,0.005040
...,...,...,...,...,...,...
9147,OAS1_0457,putamen,7960.800927,2.526669e+03,50.265981,0.006314
9148,OAS1_0457,temporal,94703.895828,3.332205e+06,1825.432770,0.019275
9149,OAS1_0457,thalamus,13499.907644,3.579510e+03,59.829004,0.004432
9150,OAS1_0457,total_cerebral_white_matter,361514.435530,9.824032e+04,313.433114,0.000867


In [114]:
grouped.to_csv("individual7_drwelly_20240430_before_final.csv")

In [67]:
#grouped.to_csv("individual_DRWELLY_before.csv")

In [115]:
roi_mean_cv = grouped.groupby('ROI')['CV'].mean().reset_index()
roi_mean_cv.rename(columns={'CV': 'Mean_CV'}, inplace=True)

roi_mean_cv


,ROI,Mean_CV
0,3rd_ventricle,0.016073
1,4th_ventricle,0.028253
2,accumbens_area,0.022256
3,amygdala,0.028161
4,brain_stem,0.010600
5,caudate,0.005285
6,cerebellum,0.013450
7,cingulate,0.009599
8,csf,0.016411
9,frontal,0.007714


In [117]:
roi_mean_cv["Mean_CV"].mean()

0.012629705391439565

In [70]:
#roi_mean_cv.to_csv("roi_mean_CV_DRwelly_before_combined_20240415.csv")

In [118]:
from scipy import stats
import numpy as np
from scipy.stats import sem, t
from numpy import sqrt

In [119]:
def calculate_mean_ci(group):

    cv_data = group['CV']
    n = len(cv_data) 
    mean_cv = cv_data.mean()  
    se = sem(cv_data, nan_policy='omit')  
    h = se * t.ppf((1 + 0.95) / 2., n-1) 

    return pd.Series({
        'Mean_CV': mean_cv,
        'CI_Lower': mean_cv - h,
        'CI_Upper': mean_cv + h
    })

In [120]:
roi_mean_ci = grouped.groupby('ROI').apply(calculate_mean_ci).reset_index().round(3)

roi_mean_ci

/tmp/ipykernel_3567064/1203373891.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  roi_mean_ci = grouped.groupby('ROI').apply(calculate_mean_ci).reset_index().round(3)


,ROI,Mean_CV,CI_Lower,CI_Upper
0,3rd_ventricle,0.016,0.015,0.017
1,4th_ventricle,0.028,0.021,0.035
2,accumbens_area,0.022,0.021,0.024
3,amygdala,0.028,0.026,0.030
4,brain_stem,0.011,0.009,0.013
5,caudate,0.005,0.005,0.006
6,cerebellum,0.013,0.009,0.018
7,cingulate,0.010,0.009,0.010
8,csf,0.016,0.015,0.017
9,frontal,0.008,0.007,0.008


In [121]:
roi_mean_ci = roi_mean_ci.round(3)

In [122]:
roi_mean_ci['Mean_CV'].mean()

0.012636363636363638

In [130]:
def calculate_session_mrd_ci(group):
    group = group.sort_values(by='Session')
    volumes = group['Volume'].values

    n = len(volumes)
    if n >= 2:  # 변경된 조건: 볼륨의 개수가 2개 이상이면 계산을 진행
        mean_volume = volumes.mean()
        num_pairs = n * (n - 1) / 2

        diffs = [abs(volumes[i] - volumes[j]) for i in range(n) for j in range(i+1, n)]
        mrd = sum(diffs) / (num_pairs * mean_volume) * 100

        se_mrd = stats.sem(diffs) / (num_pairs * mean_volume) * 100
        ci_95_mrd = stats.t.interval(0.95, df=len(diffs)-1, loc=mrd, scale=se_mrd)

        return pd.Series({'MRD': mrd, 'MRD_CI_Lower': ci_95_mrd[0], 'MRD_CI_Upper': ci_95_mrd[1]})
    else:
        return pd.Series({'MRD': pd.NA, 'MRD_CI_Lower': pd.NA, 'MRD_CI_Upper': pd.NA})
   
# 데이터프레임 예시 및 함수 실행
final = pd.DataFrame(all_data_long)
mrd_results = final.groupby(['Subject_ID', 'ROI']).apply(calculate_session_mrd_ci).reset_index()

/tmp/ipykernel_3567064/443891702.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  mrd_results = final.groupby(['Subject_ID', 'ROI']).apply(calculate_session_mrd_ci).reset_index()


In [131]:
mrd_results 

,Subject_ID,ROI,MRD,MRD_CI_Lower,MRD_CI_Upper
0,OAS1_0001,3rd_ventricle,1.114436,1.007085,1.221787
1,OAS1_0001,4th_ventricle,2.772801,2.430588,3.115014
2,OAS1_0001,accumbens_area,2.980816,2.662990,3.298642
3,OAS1_0001,amygdala,3.185651,2.907208,3.464094
4,OAS1_0001,brain_stem,0.600992,0.527565,0.674419
...,...,...,...,...,...
9147,OAS1_0457,putamen,0.841615,0.538961,1.144270
9148,OAS1_0457,temporal,2.273775,0.748985,3.798566
9149,OAS1_0457,thalamus,0.535253,0.204569,0.865937
9150,OAS1_0457,total_cerebral_white_matter,0.114993,0.071842,0.158144


In [132]:
roi_grouped = mrd_results.groupby('ROI').agg({
    'MRD': 'mean',
    'MRD_CI_Lower': 'mean',
    'MRD_CI_Upper': 'mean'
}).reset_index().round(3)

In [133]:
roi_grouped

,ROI,MRD,MRD_CI_Lower,MRD_CI_Upper
0,3rd_ventricle,1.982,1.632,2.331
1,4th_ventricle,3.424,2.878,3.971
2,accumbens_area,2.726,2.244,3.208
3,amygdala,3.465,2.861,4.069
4,brain_stem,1.296,1.084,1.509
5,caudate,0.649,0.543,0.754
6,cerebellum,1.641,1.383,1.899
7,cingulate,1.166,0.955,1.377
8,csf,2.008,1.659,2.357
9,frontal,0.928,0.763,1.093


In [127]:
roi_grouped['MRD'].mean()

1.5742884204536454

In [134]:
mrd_results.to_csv('20240430_group_mrd_drwelly_before_final_111.csv')

In [129]:
pwd

'/home/limseoyoung/debug'